# Working notebook for documents labelling
to be done:
- words cloud for each tag subset

In [71]:
import pandas as pd
import os
TEXTS_PATH = os.path.join(os.path.dirname(os.getcwd()), "src", "text_extraction", "txt_selection")
RESPONSES_PATH = os.path.join(os.path.dirname(os.getcwd()), "src", "scraping", "output", "responses_2018_now.json")

pd.set_option('display.max_rows', 10)

In [72]:
#retrieving the names of all selected files
TEXTS_PATH
txt_filenames = [f for f in os.listdir(TEXTS_PATH) if f.endswith(".txt")]

In [11]:
#reading json file to retrieve tags
responses = pd.read_json(RESPONSES_PATH)
#https://raw.githubusercontent.com/naomithiru/KPMG-Usecase/philippe/src/scraping/output/responses_2018_now.json

Index(['jcId', 'jcFr', 'jcNl', 'titleFr', 'titleNl', 'themesFr', 'themesNl',
       'signatureDate', 'validityDate', 'depositDate', 'recordDate',
       'depositRegistrationDate', 'depositNumber', 'enforced',
       'royalDecreeDate', 'noticeDepositMBDate', 'publicationRoyalDecreeDate',
       'correctedDate', 'documentLink', 'documentSize', 'scopeFr', 'scopeNl',
       'noScopeFr', 'noScopeNl'],
      dtype='object')

In [79]:
#filtering only relevant columns
columns = ['jcId', 'jcFr', 'titleFr', 'themesFr', 'scopeFr', 'noScopeFr', 'documentLink']
responses2 = responses.copy(deep=True).loc[:,columns]
#getting txt_name from original pdf name
responses2["txt_name"] = responses2["documentLink"]
responses2["txt_name"] = responses2["txt_name"].str.replace("/","-")
responses2["txt_name"] = responses2["txt_name"].str.replace(".pdf","_FR.txt")
#filtering json only for selected files and columns
columns += ['txt_name']
responses2 = responses2.loc[responses2.txt_name.isin(txt_filenames), columns]
responses2.head(2)

t:\users\fra\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:7: FutureWarning: The default value of regex will change from True to False in a future version.
  import sys


,jcId,jcFr,titleFr,themesFr,scopeFr,noScopeFr,documentLink,txt_name
34,1110000,COMMISSION PARITAIRE DES CONSTRUCTIONS METALLI...,modification du régime de pension sectoriel so...,[PENSIONS COMPÉMENTAIRES ET ASSURANCES GROUPES],None,[les employeurs et ouvriers des entreprises ex...,111/111-2018-013525.pdf,111-111-2018-013525_FR.txt
35,1110000,COMMISSION PARITAIRE DES CONSTRUCTIONS METALLI...,allocation spéciale compensatoire,[PRIME SYNDICALE],None,None,111/111-2018-012196.pdf,111-111-2018-012196_FR.txt


In [81]:
#generating all possible FR themes
themes_fr = []
for r in  responses2.themesFr:
    if r is not None:
        for t in r:
            if t not in themes_fr:
                themes_fr += [t]
themes_fr

['PENSIONS COMPÉMENTAIRES ET ASSURANCES GROUPES',
 'PRIME SYNDICALE',
 'TRAVAIL EN ÉQUIPE ET DE NUIT',
 'PÉCULE DE VACANCES',
 "PRIME PROPRE AU SECTEUR OU À L'ENTREPRISE",
 'REMBOURSEMENT DE FRAIS (HORS FRAIS DE DÉPLACEMENT)',
 'DÉTACHEMENT',
 "PRIME DE FIN D'ANNÉE",
 'SALAIRES',
 'JEUNES TRAVAILLEURS: SALAIRES',
 'FORMATION (EXCL. FORMATION SYNDICALE)',
 'TRAVAILLEURS AGÉS-EXCL.PENSIONS COMPL, PRÉPENSION(RCC), CRÉDIT-TEMPS',
 'MALADIE/ACCIDENT/DÉCÈS',
 'CHÔMAGE (AUTRE QUE LE CHÔMAGE ÉCONOMIQUE DES EMPLOYÉS)',
 "RÉGIME DE CHÔMAGE AVEC COMPLÉMENT D'ENTREPRISE (RCC)",
 'FORMATION SYNDICALE',
 "FONDS DE SÉCURITÉ D'EXISTENCE",
 'INDEMNITÉS POUR FRAIS DE DÉPLACEMENT',
 'CRÉDIT-TEMPS/DIMINUTION DE CARRIÈRE, EMPLOI FIN DE CARRIÈRE',
 'GROUPES À RISQUE',
 'TRAVAIL TEMPORAIRE ET TRAVAIL INTÉRIMAIRE',
 'RECRUTEMENT',
 'BIEN-ÊTRE AU TRAVAIL',
 'PRÉAVIS/LICENCIEMENT',
 'TEMPS DE TRAVAIL EN HEURES',
 'CHÈQUES-REPAS',
 'ECOCHÈQUES',
 'JOUR DE CONGÉ PAYÉ (PAS DE JOUR RTT OU DE COMPENSATION) ET JOUR F

In [92]:
#code to retrieve the entire text body
#for filename in responses2.txt_name.to_list():
filename = responses2.txt_name.to_list()[0]
file_path = os.path.join(TEXTS_PATH, filename)
with open(file_path, 'r', encoding="utf8") as f:
    #to retrieve entire body not necessary
    #responses2.loc[responses2.txt_name == filename, "txt_body"] = f.read()

    line_list = f.readlines()
    whole_text = ""
    for line in line_list:
        whole_text += line
    ch_list = whole_text.lower().split("chapitre")
    #responses2.loc[responses2.txt_name == filename, "chapters"] = ch_list

In [93]:
ch_list

['neerlegging-dépôt: 18/12/2018 regist.-enregistr.: 07/01/2019 n°: 149887/c0/111\n\ncommission paritaire des constructions | metallique, mecanique et electrique (pc | |\n\nconvention collective de travail du 17 decembre 2018 modifiant le regime de pension sectoriel social, le reglement\n\nde pension et le reglement de\n\nsolidarite | ',
 " 1 - champ d'application | | article 1 | |\n\n$1. la présente convention s'applique aux employeurs et ouvriers des entreprises ressortissant | à la commission paritaire 111 des constructions; métallique, mécanique et électrique, à l'exclusion des employeurs et ouvriers des entreprises exemptées du paiement d’une cotisation pour le régime de pension sectoriel, sur la base d'un accord d'entreprise relatif à l'instauration ou à l'élargissement d'un régime de pension complémentaire, conclu au plus tard le 31: décembre 1999, et pour autant que ce régime ait été | approuvé par le fonds de sécurité d'existence des fabrications métalliques (créé par décision 

In [98]:
responses2.loc[responses2.txt_name == filename, "chapters"] = [c for c in ch_list]

ValueError: Length of values (65) does not match length of index (520)

In [83]:
#testing code for filter depending on theme fr
#for t in themes:
t = themes_fr[0]
mask = responses2.themesFr.apply(lambda x: t in x)
r_t = responses2[mask]
print(t)
print(r_t)

ValueError: Must have equal len keys and value when setting with an iterable

# NLP to be implemented

In [ ]:
from src.information_extraction.nlp_preprocess import Preprocess
preprocess = Preprocess(nlp_model='en_core_web_md')

t = preprocess.clean_text(text_sample)

In [ ]:
responses2.loc[:, "txt_cleaned"] =  responses2.loc[:, "txt_body"].apply(preprocess.clean_text)
